## Imports and Setup

In [1]:
import os
import sys
import joblib
import subprocess

import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm.notebook import tqdm

sys.path.insert(0, "../utils/")
import pred_utils
import config
import geoutils
import eval_utils
import fusion_utils

import logging
logging.basicConfig(level=logging.INFO)
pd.set_option('mode.chained_assignment', None)

%load_ext autoreload
%autoreload 2

## Load Dataset

In [2]:
config_file = '../configs/datagen/datagen_COLIHAUT.yaml'
c = config.create_config(config_file, prefix='.')
bldgs = geoutils.get_annotated_bldgs(c)
#bldgs = bldgs[bldgs.shape_name == 'Saint Peter']
bldgs.head(3)

INFO:root:Dimensions: (522, 5)
INFO:root:Dimensions (non-null): (373, 5)
INFO:root:roof_material
HEALTHY_METAL      187
IRREGULAR_METAL    130
CONCRETE_CEMENT     46
INCOMPLETE          10
Name: count, dtype: int64
INFO:root:roof_type
GABLE      247
FLAT        72
HIP         44
NO_ROOF     10
Name: count, dtype: int64


,UID,roof_type,roof_material,roof_condition,geometry
0,34856,NaN,NaN,NaN,"POLYGON Z ((664865.095 1712450.406 3.039, 6648..."
1,34869,NaN,NaN,NaN,"POLYGON Z ((664898.239 1712467.912 4.035, 6648..."
2,34871,NaN,NaN,NaN,"POLYGON Z ((664921.622 1712474.439 5.993, 6649..."


## Generate Post-disaster Predictions

In [8]:
in_file = '../data/rasters/ortho/ortho_DOM.tif'
exp_config = '../configs/cnn/cnn_11.yaml'
bldgs = pred_utils.predict(bldgs, in_file, exp_config, prefix='.')
bldgs.head(3)

INFO:root:Config: {'rasters_dir': '../data/rasters/', 'csv_dir': '../data/csv/', 'exp_dir': '../exp/', 'version': 'version_02', 'attribute': 'roof_material', 'exp_name': 'exp_11', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'resnet50', 'resampler': 'RandomOverSampler', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'patience': 7, 'dropout': 0.5, 'momentum': None, 'gamma': None, 'step_size': None}
INFO:root:Model file ../exp/version_02\exp_11\best_model.pth successfully loaded.
100%|███████████████| 522/522 [04:47<00:00,  1.82it/s]                                                                 


,UID,roof_type,roof_material,roof_condition,geometry,BLUE_TARP_PROB,CONCRETE_CEMENT_PROB,HEALTHY_METAL_PROB,INCOMPLETE_PROB,IRREGULAR_METAL_PROB
0,34856,NaN,CONCRETE_CEMENT,NaN,"POLYGON Z ((664865.095 1712450.406 3.039, 6648...",0.018568,0.934198,0.014461,0.012563,0.020210
1,34869,NaN,HEALTHY_METAL,NaN,"POLYGON Z ((664898.239 1712467.912 4.035, 6648...",0.022339,0.029571,0.907237,0.014723,0.026130
2,34871,NaN,CONCRETE_CEMENT,NaN,"POLYGON Z ((664921.622 1712474.439 5.993, 6649...",0.014157,0.918398,0.015416,0.036072,0.015957


In [10]:
in_file = '../data/rasters/lidar/ndsm_DOM.tif'
exp_config = '../configs/cnn/cnn_05.yaml'
bldgs = pred_utils.predict(bldgs, in_file, exp_config, prefix='.')
bldgs.head(3)

INFO:root:Config: {'data_dir': '../data/rasters/tiles/', 'csv_dir': '../data/csv/', 'exp_dir': '../exp/', 'version': 'version_01', 'attribute': 'roof_type', 'mode': 'LIDAR', 'exp_name': 'exp_05', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'resnet50', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': 0.5}
INFO:root:Model file ../exp/version_01\exp_05\best_model.pth successfully loaded.
100%|███████████████| 522/522 [04:45<00:00,  1.83it/s]                                                                 


,UID,roof_type,roof_material,roof_condition,geometry,BLUE_TARP_PROB,CONCRETE_CEMENT_PROB,HEALTHY_METAL_PROB,INCOMPLETE_PROB,IRREGULAR_METAL_PROB,FLAT_PROB,GABLE_PROB,HIP_PROB,NO_ROOF_PROB
0,34856,FLAT,CONCRETE_CEMENT,NaN,"POLYGON Z ((664865.095 1712450.406 3.039, 6648...",0.018568,0.934198,0.014461,0.012563,0.020210,0.997483,0.001140,0.000552,0.000825
1,34869,GABLE,HEALTHY_METAL,NaN,"POLYGON Z ((664898.239 1712467.912 4.035, 6648...",0.022339,0.029571,0.907237,0.014723,0.026130,0.001214,0.996188,0.002186,0.000412
2,34871,NO_ROOF,CONCRETE_CEMENT,NaN,"POLYGON Z ((664921.622 1712474.439 5.993, 6649...",0.014157,0.918398,0.015416,0.036072,0.015957,0.059678,0.014156,0.008419,0.917748


In [4]:
out_path = f"../output/{c['version']}"
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'Colihaut_rgb_20230626.gpkg')
bldgs.to_file(out_file, driver='GPKG')

## Generate Pre-disaster Predictions

In [5]:
in_file = '../data/rasters/drone/drone_colihaut_DOM.tif'
exp_config = '../configs/cnn/cnn_11.yaml'
bldgs = pred_utils.predict(bldgs, in_file, exp_config, prefix='.')
bldgs.head(3)

Config: {'rasters_dir': '../data/rasters/', 'csv_dir': '../data/csv/', 'exp_dir': '../exp/', 'version': 'version_02', 'attribute': 'roof_material', 'exp_name': 'exp_11', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'resnet50', 'resampler': 'RandomOverSampler', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'label_smoothing': 0.1, 'lr': 1e-05, 'img_size': 224, 'patience': 7, 'dropout': 0.5, 'momentum': None, 'gamma': None, 'step_size': None}


INFO:root:Model file ../exp/version_02\exp_11\best_model.pth successfully loaded.
100%|███████████████| 522/522 [06:40<00:00,  1.30it/s]                                                                 


,UID,roof_type,roof_material,roof_condition,geometry,BLUE_TARP_PROB,CONCRETE_CEMENT_PROB,HEALTHY_METAL_PROB,INCOMPLETE_PROB,IRREGULAR_METAL_PROB,BLUE_TARP_PROB,CONCRETE_CEMENT_PROB,HEALTHY_METAL_PROB,INCOMPLETE_PROB,IRREGULAR_METAL_PROB
0,34856,NaN,CONCRETE_CEMENT,NaN,"POLYGON Z ((664865.095 1712450.406 3.039, 6648...",0.018568,0.934198,0.014461,0.012563,0.020210,0.018733,0.850857,0.108882,0.010298,0.011230
1,34869,NaN,HEALTHY_METAL,NaN,"POLYGON Z ((664898.239 1712467.912 4.035, 6648...",0.022339,0.029571,0.907237,0.014723,0.026130,0.033398,0.036099,0.877389,0.018740,0.034373
2,34871,NaN,INCOMPLETE,NaN,"POLYGON Z ((664921.622 1712474.439 5.993, 6649...",0.014157,0.918398,0.015416,0.036072,0.015957,0.072086,0.016228,0.053231,0.551563,0.306892


In [4]:
in_file = '../data/rasters/Drone/drone_colihaut_DOM.tif'
exp_config = '../configs/config_04.yaml'
bldgs = pred_utils.predict(bldgs, in_file, exp_config, prefix='.')
bldgs.head(3)

Config: {'data_dir': './data/rasters/Tiles/ortho/', 'csv_dir': './data/csv/', 'attribute': 'roof_type', 'exp_name': 'exp-04', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'inceptionv3', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 299, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None}
Model file ../exp/exp-04/best_model.pth successfully loaded.


100%|███████████████| 373/373 [06:07<00:00,  1.01it/s]                                                                 


,UID,roof_type,roof_material,geometry
4,34876,NO_ROOF,CONCRETE_CEMENT,"POLYGON Z ((665060.888 1712479.780 35.786, 665..."
5,34887,GABLE,HEALTHY_METAL,"POLYGON Z ((665078.259 1712486.585 32.366, 665..."
6,34890,GABLE,HEALTHY_METAL,"POLYGON Z ((665070.715 1712492.135 27.175, 665..."


In [13]:
in_file = '../data/rasters/DSM/ndsm_DOM.tif'
exp_config = '../configs/config_05.yaml'
bldgs = pred_utils.predict(bldgs, in_file, exp_config, prefix='.')
bldgs.head(3)

Config: {'data_dir': './data/rasters/Tiles/ndsm/', 'csv_dir': './data/csv/', 'attribute': 'roof_type', 'mode': 'GRAYSCALE', 'exp_name': 'exp-05', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'resnet50', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': 0.5}
Model file ../exp/exp-05/best_model.pth successfully loaded.


100%|███████████████| 1272/1272 [11:39<00:00,  1.82it/s]                                                               


,UID,Elevation,Layer,roof_type,roof_material,roof_condition,shape_name,geometry
34411,34413,17.535746,BLD_GEN_OUTLINE,HIP,CONCRETE_CEMENT,NaN,Saint Peter,MULTIPOLYGON Z (((665583.468 1710787.345 14.89...
34476,34478,5.934477,BLD_GEN_OUTLINE,GABLE,CONCRETE_CEMENT,NaN,Saint Peter,MULTIPOLYGON Z (((665565.273 1710859.963 5.669...
34487,34489,6.699960,BLD_RESIDENTIAL_TRAILER,FLAT,CONCRETE_CEMENT,NaN,Saint Peter,MULTIPOLYGON Z (((665534.946 1710892.122 6.222...


In [6]:
name = 'Colihaut'
out_path = '../output/'
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}-drone-20230616.gpkg')
bldgs.to_file(out_file, driver='GPKG')

## Model Evaluation

In [19]:
ytrue_file = '../data/vectors/building_footprints_annotated_COLIHAUT.gpkg'
ytrue = gpd.read_file(ytrue_file)[['UID', 'roof_type', 'roof_material', 'geometry']].dropna()
ytrue.roof_type = ytrue.roof_type.replace({'PYRAMID': 'HIP', 'HALF_HIP': 'HIP'})
print(f"Data dimensions: {ytrue.shape}")
print(ytrue.roof_type.value_counts())
print(ytrue.roof_material.value_counts())
ytrue.head(3)

Data dimensions: (373, 4)
roof_type
GABLE      247
FLAT        72
HIP         44
NO_ROOF     10
Name: count, dtype: int64
roof_material
HEALTHY_METAL      187
IRREGULAR_METAL    130
CONCRETE_CEMENT     46
INCOMPLETE          10
Name: count, dtype: int64


,UID,roof_type,roof_material,geometry
4,34876,NO_ROOF,INCOMPLETE,"POLYGON Z ((665060.888 1712479.780 35.786, 665..."
5,34887,GABLE,HEALTHY_METAL,"POLYGON Z ((665078.259 1712486.585 32.366, 665..."
6,34890,GABLE,IRREGULAR_METAL,"POLYGON Z ((665070.715 1712492.135 27.175, 665..."


In [18]:
ypred_file = f'../output/{name}-drone-20230616.gpkg'
ypred = gpd.read_file(ypred_file)[['UID', 'roof_type', 'roof_material', 'geometry']].dropna()
print(f"Data dimensions: {ypred.shape}")
print(ypred.roof_type.value_counts())
print(ypred.roof_material.value_counts())
ypred.head(3)

Data dimensions: (373, 4)
roof_type
GABLE      250
HIP         59
FLAT        54
NO_ROOF     10
Name: count, dtype: int64
roof_material
HEALTHY_METAL      204
IRREGULAR_METAL    113
CONCRETE_CEMENT     48
INCOMPLETE           7
BLUE_TARP            1
Name: count, dtype: int64


,UID,roof_type,roof_material,geometry
0,34876,NO_ROOF,CONCRETE_CEMENT,"POLYGON Z ((665060.888 1712479.780 35.786, 665..."
1,34887,GABLE,HEALTHY_METAL,"POLYGON Z ((665078.259 1712486.585 32.366, 665..."
2,34890,GABLE,HEALTHY_METAL,"POLYGON Z ((665070.715 1712492.135 27.175, 665..."


In [26]:
attribute = 'roof_type'
classes = geoutils.classes_dict[attribute]
cm = eval_utils.get_confusion_matrix(ytrue[attribute], ypred[attribute], classes)
eval_utils.evaluate(ytrue[attribute], ypred[attribute])

{'overall_accuracy': 89.27613941018767,
 'balanced_accuracy': 87.45253649858913,
 'f1_score_micro': 89.27613941018767,
 'f1_score': 86.36588137237665,
 'precision_score': 87.00772128060264,
 'recall_score': 87.45253649858913,
 'f1_per_class': array([80.95238095, 92.95774648, 81.55339806, 90.        ]),
 'precision_per_class': array([94.44444444, 92.4       , 71.18644068, 90.        ]),
 'recall_per_class': array([70.83333333, 93.52226721, 95.45454545, 90.        ])}

## Data Fusion

In [8]:
c = config.create_config("../configs/fusion_06.yaml")
c = {key: '.' + value if 'config' in key or 'dir' in key else value for key, value in c.items()}

c1 = config.create_config(c['config1'])
c1 = {key: '.' + value if 'dir' in key else value for key, value in c1.items()}
classes = geoutils.classes_dict[c1['attribute']]
exp_dir = os.path.join(c['exp_dir'], c1['exp_name'])
model1 = pred_utils.load_model(c1, exp_dir=exp_dir , n_classes=len(classes))
print(c1)

c2 = config.create_config(c['config2'])
exp_dir = os.path.join(c['exp_dir'], c2['exp_name'])
c2 = {key: '.' + value if 'dir' in key else value for key, value in c2.items()}
model2 = pred_utils.load_model(c2, exp_dir=exp_dir, n_classes=len(classes))
print(c2)

Model file ../exp/exp-09\best_model.pth successfully loaded.
{'data_dir': '../data/rasters/Tiles/ortho/', 'csv_dir': '../data/csv/', 'attribute': 'roof_material', 'exp_name': 'exp-09', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'efficientnetb0', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None}
Model file ../exp/exp-06\best_model.pth successfully loaded.
{'data_dir': '../data/rasters/Tiles/ndsm/', 'csv_dir': '../data/csv/', 'attribute': 'roof_material', 'exp_name': 'exp-06', 'mode': 'GRAYSCALE', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'inceptionv3', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 299, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None}


In [101]:
source1 = '../data/rasters/Ortho/ortho_DOM.tif'
source2 = '../data/rasters/DSM/ndsm_DOM.tif'
data = fusion.predict(bldgs, c1, c2, model1, model2, source1=source1, source2=source2)

100%|███████████████| 522/522 [11:29<00:00,  1.32s/it]                                                                 


In [9]:
features = fusion.get_features(c, data)
model_file = os.path.join(c['exp_dir'], c['exp_name'], c['mode'], c['model'], 'best_model.pkl')
model = joblib.load(model_file)
preds = model.predict(data[features])

Pipeline(steps=[('scaler', MinMaxScaler()), ('selector', None),
                ('model',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
                                        max_depth=9, n_estimators=450,
                                        n_jobs=-1, random_state=42,
                                        verbose=1))])

In [104]:
bldgs['roof_material'] = preds
out_path = '../output/'
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}-ortho-20230613.gpkg')
bldgs.to_file(out_file, driver='GPKG')